# Introdução

## Importação das bibliotecas necessarias

In [1]:
%matplotlib inline
import pandas as pd
import subprocess
import requests
import json
import sys
import os
import re

## Funções auxiliares

In [2]:
def return_sql_query(query_string):
    ret = {
        "token": {
            "token": token
        },
        "sql": {
            "sql": {
                "query":query_string
            }
        }
    }
    return ret

def json_to_df(resp):
    columns = [col['name'] for col in resp.json()['columns']]
    values = resp.json()['rows']  
    return pd.DataFrame(values, columns=columns)

def rows_and_columns_to_df(columns, rows):
    columns_array = [col['name'] for col in columns]
    return pd.DataFrame(rows, columns=columns_array)

def fetch_from_tabnet(year, tipo):
    if tipo == 'materno':
        incremento = '%D3bitos_maternos'
    elif tipo == 'fertil':
        incremento = '%D3bitos_mulheres_idade_f%E9rtil'

    command = "curl -X POST 'http://tabnet.datasus.gov.br/cgi/tabcgi.exe?sim/cnv/mat10br.def' --data 'Linha=Munic%EDpio&Coluna=%D3bito_investigado&Incremento="+incremento+"&Arquivos=matbr"+year+".dbf&pesqmes1=Digite+o+texto+e+ache+f%E1cil&SMunic%EDpio=TODAS_AS_CATEGORIAS__&pesqmes2=Digite+o+texto+e+ache+f%E1cil&SCapital=TODAS_AS_CATEGORIAS__&pesqmes3=Digite+o+texto+e+ache+f%E1cil&SRegi%E3o_de_Sa%FAde_%28CIR%29=TODAS_AS_CATEGORIAS__&pesqmes4=Digite+o+texto+e+ache+f%E1cil&SMacrorregi%E3o_de_Sa%FAde=TODAS_AS_CATEGORIAS__&pesqmes5=Digite+o+texto+e+ache+f%E1cil&SMicrorregi%E3o_IBGE=TODAS_AS_CATEGORIAS__&pesqmes6=Digite+o+texto+e+ache+f%E1cil&SRegi%E3o_Metropolitana_-_RIDE=TODAS_AS_CATEGORIAS__&pesqmes7=Digite+o+texto+e+ache+f%E1cil&STerrit%F3rio_da_Cidadania=TODAS_AS_CATEGORIAS__&pesqmes8=Digite+o+texto+e+ache+f%E1cil&SMesorregi%E3o_PNDR=TODAS_AS_CATEGORIAS__&SAmaz%F4nia_Legal=TODAS_AS_CATEGORIAS__&SSemi%E1rido=TODAS_AS_CATEGORIAS__&SFaixa_de_Fronteira=TODAS_AS_CATEGORIAS__&SZona_de_Fronteira=TODAS_AS_CATEGORIAS__&SMunic%EDpio_de_extrema_pobreza=TODAS_AS_CATEGORIAS__&pesqmes14=Digite+o+texto+e+ache+f%E1cil&SCap%EDtulo_CID-10=TODAS_AS_CATEGORIAS__&pesqmes15=Digite+o+texto+e+ache+f%E1cil&SGrupo_CID-10=TODAS_AS_CATEGORIAS__&pesqmes16=Digite+o+texto+e+ache+f%E1cil&SCategoria_CID-10=TODAS_AS_CATEGORIAS__&STipo_causa_obst%E9tr=TODAS_AS_CATEGORIAS__&pesqmes18=Digite+o+texto+e+ache+f%E1cil&SFaixa_Et%E1ria=TODAS_AS_CATEGORIAS__&pesqmes19=Digite+o+texto+e+ache+f%E1cil&SFaixa_Et%E1ria_OPS=TODAS_AS_CATEGORIAS__&pesqmes20=Digite+o+texto+e+ache+f%E1cil&SFaixa_Et%E1ria_det=TODAS_AS_CATEGORIAS__&SCor%2Fra%E7a=TODAS_AS_CATEGORIAS__&SEscolaridade=TODAS_AS_CATEGORIAS__&SEstado_civil=TODAS_AS_CATEGORIAS__&SLocal_ocorr%EAncia=TODAS_AS_CATEGORIAS__&SMorte_grav%2Fpuerp=TODAS_AS_CATEGORIAS__&S%D3bito_investigado=TODAS_AS_CATEGORIAS__&formato=prn&mostre=Mostra'"
    raw_data = subprocess.check_output(command, shell=True)
    pro_data_1 = re.split('PRE>', raw_data.decode("iso-8859-1"))
    if len(pro_data_1) > 1:
        pro_data_2 = pro_data_1[1]
    else:
        pro_data_2 = ''
    pro_data_3 = pro_data_2.split('\n')
    pro_data_4 = pro_data_3[2:-3]
    pro_data_5 = []
    line = ''
    for i in pro_data_4:
        line = i
        line = line.split('\r')[0]
        col = line.split('\"')
        cod_mun_res = re.search('[0-9]{6}', col[1])
        if cod_mun_res != None:
            col[1] = cod_mun_res.group()
        line = ''.join(col)
        line = line.replace('-', '0')
        line = line.split(";")
        line.append(((int(line[1])+int(line[2]))/int(line[4]))*100)
        line.insert(1, '20'+year)
        pro_data_5.append(line)
    return pro_data_5


# Proporção de óbitos de mulher em idade fértil investigados (SIM)

## Pegar dados do Tabnet

In [3]:
year_data = []
for i in range(9):
    year_data.extend(fetch_from_tabnet(str(12+i).zfill(2), 'materno'))

In [4]:
df=pd.DataFrame(year_data)
df.head()
df=df.rename(columns = {0:'CODMUNRES'})
df=df.rename(columns = {1:'Ano'})
# Óbito investigado, com ficha síntese informada
df=df.rename(columns = {2:'OICFSI'})
# Óbito investigado, sem ficha síntese informada
df=df.rename(columns = {3:'OISFSI'})
# Óbito não investigado
df=df.rename(columns = {4:'ONI'})
df=df.rename(columns = {5:'Total'})
df=df.rename(columns = {6:'Indicador'})
df=df.sort_values(by=['CODMUNRES','Ano']).reset_index(drop=True)

In [5]:
df.loc[df['CODMUNRES'] == '330455']
#df.head()

,CODMUNRES,Ano,OICFSI,OISFSI,ONI,Total,Indicador
5329,330455,2012,56,3,0,59,100.0
5330,330455,2013,67,0,0,67,100.0
5331,330455,2014,64,0,0,64,100.0
5332,330455,2015,65,0,0,65,100.0
5333,330455,2016,62,0,0,62,100.0
5334,330455,2017,70,0,0,70,100.0
5335,330455,2018,50,0,0,50,100.0
5336,330455,2019,62,0,0,62,100.0
5337,330455,2020,83,0,0,83,100.0


In [6]:
%time
df.to_csv('../csv_files/W-25-proporcao_obitos_maternos_declarados_investigados.csv',index=False)

CPU times: user 1e+03 ns, sys: 1 µs, total: 2 µs
Wall time: 6.44 µs
